<a href="https://colab.research.google.com/github/bxck75/A1_Colabs/blob/master/simpsons_newpix2pix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -r /content/*

In [ ]:

######################################################
import os 
!curl https://raw.githubusercontent.com/bxck75/colab_starter/main/setup.py -o /content/setup.py
    
######################################################

In [ ]:
!python /content/setup.py

In [ ]:
!pip install tensorflow tensorflow-addons

In [ ]:
import os
os.chdir('/content/GitGo/piss-ant-pix2pix')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/GitGo/datasets')

In [ ]:
!unzip /content/drive/MyDrive/garbage.zip

In [ ]:
!rm -r /tmp/preprocess/*

In [ ]:
%cd /content/GitGo/piss-ant-pix2pix
# resize
!python tools/process_custom.py --size 128 --pad --input_dir /content/GitGo/datasets/simpsons_cropped --operation resize --output_dir /tmp/preprocess/_resized
# blank clones
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --operation blank --output_dir /tmp/preprocess/_blank
# edge clones
#!python tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edge
#!python tools/process_custom.py --input_dir /tmp/preprocess/_edge --operation blank --output_dir /tmp/preprocess/_blank_edge
# combine
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edge --operation combine --output_dir  /tmp/preprocess/_resized/_combined_edge
#!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_resized/_combined_blank_edge
# split
!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_edge
#!python tools/split.py --dir /tmp/preprocess/_resized/_combined_blank_edge
# make outdirs
!mkdir -p /content/GitGo/datasets/simpsons_blank_dataset
#!mkdir -p /content/GitGo/datasets/simpsons_edge_dataset
#!mkdir -p /content/GitGo/datasets/simpsons_blank_edge_dataset
# copy to final folders
!cp -rf /tmp/preprocess/_resized/_combined_blank/ /content/GitGo/datasets/simpsons_blank_dataset
#!cp -rf /tmp/preprocess/_resized/_combined_edge/ /content/GitGo/datasets/simpsons_edge_dataset
#!cp -rf /tmp/preprocess/_resized/_combined_blank_edge/ /content/GitGo/datasets/simpsons_blank_edge_dataset

In [ ]:
#!python /content/GitGo/piss-ant-pix2pix/tools/edge_custom.py --input_dir /tmp/preprocess/_resized --output_dir /tmp/preprocess/_edge

In [ ]:
%cd /content/GitGo/piss-ant-pix2pix
import cv2,glob
from google.colab.patches import cv2_imshow

def edge(_in,_out):
    image = cv2.imread(_in)
    #cv2_imshow(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    # show the original and blurred images
    #cv2_imshow(image)
    #cv2_imshow( blurred)
    # compute a "wide", "mid-range", and "tight" threshold for the edges
    # using the Canny edge detector
    #wide = cv2.Canny(blurred, 10, 200)
    mid = cv2.Canny(blurred, 30, 150)
    #tight = cv2.Canny(blurred, 240, 250)
    # show the output Canny edge maps
    #cv2_imshow(~wide)
    #cv2_imshow(~mid)
    #cv2_imshow(~tight)
    cv2.imwrite(_out, ~mid)


for img_path in glob.glob('/tmp/preprocess/_resized/*.png'):
    print(img_path)
    new_img_path = img_path.replace('_resized','_edges')
    print(new_img_path)
    edge(img_path, new_img_path)


!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_edges --operation combine --output_dir  /tmp/preprocess/_combined_edge
!python tools/split.py --dir /tmp/preprocess/_combined_edge
!mkdir -p /content/GitGo/datasets/simpsons_edge_dataset
!cp -rf /tmp/preprocess/_combined_edge/ /content/GitGo/datasets/simpsons_edge_dataset

In [ ]:

!python tools/process_custom.py --input_dir /tmp/preprocess/_edges --operation blank --output_dir /tmp/preprocess/_blank_edge
!python tools/process_custom.py --input_dir /tmp/preprocess/_resized --b_dir  /tmp/preprocess/_blank_edge --operation combine --output_dir  /tmp/preprocess/_combined_blank_edge
!python tools/split.py --dir /tmp/preprocess/_combined_blank_edge
!mkdir -p /content/GitGo/datasets/simpsons_blank_edge_dataset
!cp -rf /tmp/preprocess/_combined_blank_edge /content/GitGo/datasets/simpsons_blank_edge_dataset

In [ ]:
import glob
files = glob.glob('/content/GitGo/datasets/simpsons_blank_dataset/_combined_blank/train/*.png')
print(len(files))

In [ ]:
!rm -r /content/drive/MyDrive/simpsons_metrics/*

In [ ]:
!pip install -U tensorboard-plugin-profil
%reload_ext tensorboard
logs_base_dir = '/content/simpsons_metrics'
os.makedirs(logs_base_dir, exist_ok=True)
%tensorboard --logdir {logs_base_dir}

In [ ]:
import os,warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0' 
warnings.filterwarnings('ignore')
os.chdir('/content/GitGo/piss-ant-pix2pix')
custom_dataset = '/content/GitGo/datasets/simpsons_blank_dataset/_combined_blank/train'
train_metrics = '/content/simpsons_metrics'
!mkdir -p {train_metrics}

# CHECK IF ITS A RETRAINING
if os.path.exists('/content/simpsons_metrics/checkpoint'):
    retrain = '--checkpoint '+ train_metrics
else:
    retrain = ''
# train
!python pix2pix_custom.py \
    --summary_freq 50 \
    --ngf 64 \
    --ndf 64 \
    --save_freq 50 \
    --display_freq 10 \
    --progress_freq 50 \
    --batch_size 64 \
    --mode train \
    --output_dir {train_metrics} \
    --max_epochs 3 \
    --input_dir {custom_dataset} \
    {retrain} \
    --which_direction BtoA

In [ ]:
!zip /content/drive/MyDrive/simpsons_metrics_backup.zip /content/simpsons_metrics